[View in Colaboratory](https://colab.research.google.com/github/stanimman/Simple/blob/master/Entity_Embedding.ipynb)

In [1]:
!apt-get install  -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
!mkdir data
!cd data
!pip install kaggle-cli
!kg download -u 'stanimman' -p '***' -c 'home-credit-default-risk'

mkdir: cannot create directory ‘data’: File exists
  Using cached https://files.pythonhosted.org/packages/67/61/710d02460bc4367ffd1f5e71cd9c031fb278f78aa0e8e32ca9dd99a2add8/kaggle-cli-0.12.13.tar.gz
  Using cached https://files.pythonhosted.org/packages/ee/e8/140ad9b5826920a8b85c187095e7725b87b913fc40243aa66dd04e9d82d6/cliff-2.8.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/5c/2e/f63ed26b51e36efa4cc22cad18187fcb0a253f756d548c96bb931f13de98/MechanicalSoup-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a0/b5/4c6995f8f259f0858f79460e6d277888f8498ce1c1a466dfbb24f06ba83f/lxml-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/7b/44/25b7283e50585f0b4156960691d951b05d061abf4a714078393e51929b30/cssselect-1.0.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/69/c2ce7e91c89dc073eb1aa74c0621c3eefbffe8216b3f9af9d3885265c01c/configparser-3.5.0.tar.gz
  Using cached https://

 done
  Stored in directory: /content/.cache/pip/wheels/58/32/92/2227fba72f0702e4168f084c6dfc2d6d3fd1904634ab61ca6a
Successfully built kaggle-cli cliff configparser PrettyTable pyperclip
downloading https://www.kaggle.com/c/home-credit-default-risk/download/sample_submission.csv.zip

sample_submission.csv.zip 100% |####################| Time: 0:00:00 176.8 KiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/application_test.csv.zip

application_test.csv.zip 100% |#####################| Time: 0:00:00   6.8 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/application_train.csv.zip

application_train.csv.zip 100% |####################| Time: 0:00:01  18.8 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/bureau.csv.zip

bureau.csv.zip 100% |###############################| Time: 0:00:01  18.4 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/bureau_balance.csv.zip

bureau_balan

previous_application.csv.zip 100% |#################| Time: 0:00:03  21.0 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/credit_card_balance.csv.zip

credit_card_balance.csv.zip 100% |##################| Time: 0:00:04  19.7 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/POS_CASH_balance.csv.zip

POS_CASH_balance.csv.zip 100% |#####################| Time: 0:00:04  25.1 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/installments_payments.csv.zip

installments_payments.csv.zip 100% |################| Time: 0:00:10  25.8 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/HomeCredit_columns_description.csv

HomeCredit_columns_description.csv 100% |###########| Time: 0:00:01  31.6 KiB/s



In [4]:
! apt install unzip


Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Archive:  application_train.csv.zip
  inflating: application_train.csv   
Archive:  application_test.csv.zip
  inflating: application_test.csv    


In [0]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

In [9]:
!unzip application_train.csv.zip
!unzip application_test.csv.zip

Archive:  application_train.csv.zip
  inflating: application_train.csv   
Archive:  application_test.csv.zip
  inflating: application_test.csv    


In [8]:
!ls

application_test.csv.zip     datalab
application_train.csv.zip    HomeCredit_columns_description.csv
bureau_balance.csv.zip	     installments_payments.csv.zip
bureau.csv.zip		     POS_CASH_balance.csv.zip
credit_card_balance.csv.zip  previous_application.csv.zip
data			     sample_submission.csv.zip


In [0]:
app_train = pd.read_csv('/content/application_train.csv')
app_test = pd.read_csv('/content/application_test.csv')

In [16]:
app_train.head(2)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,1,0,0,0,0,0,0,1,0


In [55]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [56]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


In [57]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 239)
Testing Features shape:  (48744, 239)


In [60]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 239)
Testing data shape:  (48744, 239)


In [61]:
val = train[246009:]
train = train[:246009]
print(train.shape)
print(val.shape)

(246009, 239)
(61502, 239)


In [53]:
print(val.shape)

(0, 239)


In [24]:
train_label = train_labels[:246009] #80% Need to randomize / Highly imbalanced
val_label = train_labels[246009:] 
print(train_label.shape)
print(val_label.shape)

(246009,)


NameError: ignored

In [0]:
#Split into Train / Validation - Not Now
#train=app_train.sample(frac=0.8,random_state=200)
#Validation=app_train.drop(train.index)

In [25]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5bc06000 @  0x7fabc2bc21c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.4.0
True


In [0]:
import torch 
import torch.nn as nn

In [0]:
# Logistic Regression for MNIST
# Hyper-parameters 
input_size = 239
num_classes = 2
num_epochs = 5
batch_size = 1000
learning_rate = 0.001

In [0]:
train_torch = torch.from_numpy(train)
train_torch = train_torch.type(torch.float)




In [0]:
val_torch = torch.from_numpy(val)
val_torch = val_torch.type(torch.float)

In [0]:
train_torch_label = train_label.astype('float32')
val_torch_label = val_label.astype('float32')


In [0]:
#train_torch_label = torch.from_numpy(np.array(train_label.astype('int32')))
#train_torch_label = train_torch_label.type(torch.FloatTensor)

In [41]:
shape_check = np.array(train_label.values.reshape(-1,1))
print(shape_check.shape)
print(shape_check.dtype)

(246009, 1)
int64


In [0]:
train_torch_label = torch.from_numpy(shape_check)
#train_torch_label = train_torch_label.type(torch.FloatTensor)


In [74]:
shape_check_en = np.array(train_label.values.reshape(-1))
print(shape_check_en.shape)
train_torch_label = torch.from_numpy(shape_check_en)
train_torch_label = train_torch_label.type(torch.LongTensor) #Entropy Loss

(246009,)


In [75]:
train_torch_label[1:100]

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
         0])

In [34]:
print(train_torch_label.shape)
print(train_torch.shape)

torch.Size([246009, 1])
torch.Size([246009, 239])


In [78]:
model = nn.Linear(input_size,2) # We try to predict the value on output (eg: House price ) with four given feature (room size, etc..)
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()  #- Not working need to know why 
optimizer = torch.optim.SGD(model.parameters(),lr=.001) # linear parameters are w, b which we get using linear.weight , linear.bias
#print ('Weight & Bias Before Training ',model.weight,'&', model.bias)
for epoch in range(1000):
  # Forward_pass
  pred = model(train_torch) # give the data to calculate forward linear layer

  loss = criterion(pred,train_torch_label) # compare prediction vs actuals
  if (epoch + 1) % 10 == 0 :
    print ('Epoch : loss ' , epoch, loss.item())

  # Make gradient to be zero
  optimizer.zero_grad()

  # Backward pass  Auto differntiation not much of an worry
  loss.backward()

  #Update using optimizer after calculating loss
  optimizer.step()

Epoch : loss  9 0.7202233672142029
Epoch : loss  19 0.656916081905365
Epoch : loss  29 0.6042614579200745
Epoch : loss  39 0.5603563189506531
Epoch : loss  49 0.523647665977478
Epoch : loss  59 0.49281832575798035
Epoch : loss  69 0.4667985439300537
Epoch : loss  79 0.44472846388816833
Epoch : loss  89 0.4259069859981537
Epoch : loss  99 0.4097803235054016
Epoch : loss  109 0.39588966965675354
Epoch : loss  119 0.3838669955730438
Epoch : loss  129 0.373418927192688
Epoch : loss  139 0.36429110169410706
Epoch : loss  149 0.35629841685295105
Epoch : loss  159 0.34924760460853577
Epoch : loss  169 0.34302955865859985
Epoch : loss  179 0.337514191865921
Epoch : loss  189 0.3326060473918915
Epoch : loss  199 0.32822930812835693
Epoch : loss  209 0.32431474328041077
Epoch : loss  219 0.32080078125
Epoch : loss  229 0.3176402747631073
Epoch : loss  239 0.31479012966156006
Epoch : loss  249 0.31221550703048706
Epoch : loss  259 0.3098870813846588
Epoch : loss  269 0.3077670931816101
Epoch : lo

Epoch : loss  739 0.2834256887435913
Epoch : loss  749 0.28334590792655945
Epoch : loss  759 0.28326845169067383
Epoch : loss  769 0.28320080041885376
Epoch : loss  779 0.2831307649612427
Epoch : loss  789 0.2830653786659241
Epoch : loss  799 0.2829993665218353
Epoch : loss  809 0.282941073179245
Epoch : loss  819 0.2828829884529114
Epoch : loss  829 0.28282636404037476
Epoch : loss  839 0.28277233242988586
Epoch : loss  849 0.28271737694740295
Epoch : loss  859 0.28267261385917664
Epoch : loss  869 0.28262361884117126
Epoch : loss  879 0.282578706741333
Epoch : loss  889 0.2825348377227783
Epoch : loss  899 0.2824908494949341
Epoch : loss  909 0.2824496328830719
Epoch : loss  919 0.2824104130268097
Epoch : loss  929 0.2823731303215027
Epoch : loss  939 0.2823340892791748
Epoch : loss  949 0.28229907155036926
Epoch : loss  959 0.28226059675216675
Epoch : loss  969 0.28222551941871643
Epoch : loss  979 0.28219348192214966
Epoch : loss  989 0.2821611166000366
Epoch : loss  999 0.28213241

In [0]:
shape_check_1 = np.array(val_label.values.reshape(-1,1))
val_torch_label = torch.from_numpy(shape_check_1)
val_torch_label = val_torch_label.type(torch.FloatTensor)

In [50]:
print(len(val_torch_label))

61502


In [52]:
val_torch.shape

torch.Size([0])

In [0]:
outputs = model(val_torch)

In [80]:
outputs.data[1:100]

tensor([[ 1.3158, -1.0888],
        [ 1.2086, -1.4148],
        [ 1.1513, -1.4100],
        [ 0.9837, -1.2315],
        [ 0.9273, -1.2824],
        [ 1.1998, -1.4108],
        [ 1.1628, -1.4402],
        [ 1.0884, -1.2363],
        [ 1.2043, -1.1194],
        [ 1.1667, -1.1946],
        [ 0.9191, -1.3910],
        [ 1.3064, -1.0397],
        [ 0.9005, -1.5179],
        [ 1.3576, -1.0511],
        [ 1.3109, -1.1683],
        [ 0.9735, -1.2716],
        [ 0.9415, -1.1160],
        [ 1.1832, -1.4745],
        [ 1.0139, -1.0569],
        [ 1.1154, -1.2072],
        [ 1.0687, -1.0962],
        [ 1.0566, -1.1157],
        [ 1.0747, -1.1220],
        [ 1.0723, -1.2843],
        [ 1.1602, -1.0342],
        [ 1.1457, -1.1542],
        [ 1.1119, -1.1272],
        [ 1.1443, -1.3893],
        [ 1.1321, -1.1698],
        [ 1.1718, -1.1324],
        [ 1.2228, -1.1434],
        [ 1.0078, -1.0492],
        [ 0.9604, -1.1373],
        [ 1.1994, -1.2910],
        [ 1.0849, -1.4739],
        [ 0.9986, -1

In [0]:
_, predicted = torch.max(outputs.data, 1)

In [82]:
predicted[1:100]

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0])

In [85]:
with torch.no_grad():
    correct = 0
    total = 61502
    outputs = model(val_torch)
    for i in range(len(val_torch_label)):
      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted[i] == val_torch_label[i].type(torch.LongTensor)).sum()
      
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
      

Accuracy of the model on the 10000 test images: 92 %


In [0]:
#_, predicted = torch.max(outputs.data, 1)
y_pred = predicted.detach().numpy()
y = val_torch_label.type(torch.LongTensor).detach().numpy()

In [0]:
from sklearn import metrics

In [97]:
y_pred.shape

(61502,)

In [98]:
fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=2)
metrics.auc(fpr, tpr)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


nan

In [0]:
aucmeter = AUCMeter()
aucmeter.add(y_pred, y)
print(aucmeter.value()[0], sklearn.metrics.roc_auc_score(y_true,y_pred))
